#### User-Level Analysis

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
#Function to Print Percentiles (for Cleaning)
def percentiles(df, variable):
    print "Min:  "+str(df[variable].quantile(q=0.00))
    print "1st:  "+str(df[variable].quantile(q=0.01))
    print "5th:  "+str(df[variable].quantile(q=0.05))
    print "10th: "+str(df[variable].quantile(q=0.10))
    print "25th: "+str(df[variable].quantile(q=0.25))
    print "50th: "+str(df[variable].quantile(q=0.50))
    print "75th: "+str(df[variable].quantile(q=0.75))
    print "90th: "+str(df[variable].quantile(q=0.90))
    print "95th: "+str(df[variable].quantile(q=0.95))
    print "99th: "+str(df[variable].quantile(q=0.99))
    print "Max:  "+str(df[variable].quantile(q=1.00))

In [4]:
user = pd.read_pickle("data/yelp/dataframes/yelp_review_user.pkl")
user.tail(100)

,location,user_id,topic_food,topic_service,topic_ambience,topic_value,total,pct_food,pct_service,pct_ambience,pct_value
21961,"Charlotte, NC",kciO7KU4MqlGwJw4soDO4w,420,133,96,42,1159,0.362381,0.114754,0.082830,0.036238
25540,"Charlotte, NC",sfrumTeq5CyNSOCF3o7UCg,482,204,78,74,1162,0.414802,0.175559,0.067126,0.063683
37335,"Madison, WI",dKm4UDJDELhM0Qo1j6GKCg,384,188,91,82,1164,0.329897,0.161512,0.078179,0.070447
2256,"Charlotte, NC",44RR-h4gBIMs0584B_XVOQ,404,95,102,53,1175,0.343830,0.080851,0.086809,0.045106
12911,"Charlotte, NC",RYkCobCYYZ2HqL6TdEqoqQ,296,169,87,65,1181,0.250635,0.143099,0.073666,0.055038
35928,"Madison, WI",XHuEbGrgPEhnBRjRUdcSgg,380,236,191,104,1197,0.317460,0.197160,0.159566,0.086884
7961,"Charlotte, NC",GZPmUmnnH7rx3xUzF5vqwQ,558,171,91,83,1204,0.463455,0.142027,0.075581,0.068937
5489,"Charlotte, NC",B5YUaH0CTFAExnV3iDa2DQ,448,298,114,78,1207,0.371168,0.246893,0.094449,0.064623
57081,"Pittsburg, PA",hkbwjnY42LDloMV84Yoy5g,558,154,122,114,1213,0.460016,0.126958,0.100577,0.093982
8692,"Charlotte, NC",I9-qaI_sfTJzeG52WZMdkQ,611,151,78,79,1220,0.500820,0.123770,0.063934,0.064754


In [7]:
percentiles(user, "total")
df = user[user.total >= 20]

Min:  1.0
1st:  1.0
5th:  2.0
10th: 3.0
25th: 5.0
50th: 9.0
75th: 20.0
90th: 46.0
95th: 85.0
99th: 365.71
Max:  7186.0


In [8]:
#Look at Pairwise Distances for Top 20 Users
df.pct_food

count    15896.000000
mean         0.380943
std          0.120078
min          0.000000
25%          0.303903
50%          0.380952
75%          0.454545
max          1.000000
Name: pct_food, dtype: float64

In [12]:
df.pct_service.describe()

count    15896.000000
mean         0.159155
std          0.093378
min          0.000000
25%          0.095238
50%          0.142857
75%          0.205128
max          0.800000
Name: pct_service, dtype: float64

In [13]:
df.pct_ambience.describe()

count    15896.000000
mean         0.103610
std          0.066976
min          0.000000
25%          0.054054
50%          0.094938
75%          0.142857
max          0.483871
Name: pct_ambience, dtype: float64

In [14]:
df.pct_value.describe()

count    15896.000000
mean         0.061065
std          0.049919
min          0.000000
25%          0.027923
50%          0.050000
75%          0.086957
max          0.416667
Name: pct_value, dtype: float64

In [ ]:
>>> import numpy as np
>>> 
>>> X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
>>> model = TSNE(n_components=2, random_state=0)
>>> np.set_printoptions(suppress=True)
>>> model.fit_transform(X) 
array([[ 0.00017599,  0.00003993],
       [ 0.00009891,  0.00021913],
       [ 0.00018554, -0.00009357],
       [ 0.00009528, -0.00001407]])